In [1]:
from bs4 import BeautifulSoup
import time
import numpy as np
import pylab as py
import pandas as pd
import html.parser
from selenium import webdriver


In [2]:
#This functions returns the url to be used to fetch data from teams
def get_team_url(team):
    base_url='https://www.whoscored.com/Teams/26/Archive/England-'
    team_url=base_url+team
    return team_url

In [3]:
'''On whoscored the archived data link has a number associated with it for each past season. This function
fetches the values that need to be fetched to go to those past team archives
'''
def get_archived_value(team_url,season,driver):
    #To open and get the page on chrome
    print(season)   #prints the season we are crawling right now
    #print(type(season))
    team_text=driver.get(team_url)     #Opens the page of the team
    soup_team = BeautifulSoup(driver.page_source, 'lxml')
    import re
    #Searches for all the tags in the drop down menu of past seasons which have premier on them and then selects the particular one for the season we are looking for
    tag_past_season=soup_team.find('option', string="Premier League - "+ season)
    print(tag_past_season)
    tag_lister=tag_past_season['value']    #gets the tag id number for the season we are looking for
    '''
    for option in data_past_seasons:
        tag_lister.append(option['value'])
    driver.delete_all_cookies()
    '''
    return tag_lister

In [4]:
'''to return the names of teams that played in a particular season of the premier league'''
def get_season_teams(season,driver):
    base_PL_url='https://www.whoscored.com/Regions/252/Tournaments/2/England-Premier-League'  #PL url
    base_url='https://www.whoscored.com'  #stats website url
    import re
    PL_page=driver.get(base_PL_url)    #opens the webiste of the premier league
    soup_PL = BeautifulSoup(driver.page_source, 'lxml')
    archived_seasons=soup_PL.find_all('option',attrs={"value":True})  #the seasons are listed in a dropdown menu
    #print(archived_seasons)
    for elements in archived_seasons:
        if elements.getText()==season:   #searches for the season you are looking for in the oprions
            print(elements['value'])
            season_tag=elements['value']  #gets the tag id number for that season
            
            break
            
    season_url=base_url+season_tag    #new URL for the season  
    season_page=driver.get(season_url)  #this takes us to the page of the season
    soup_season=BeautifulSoup(driver.page_source, 'lxml')   
    season_table=soup_season.find_all('tr' ,attrs={'data-team-id':True})   #get all the team ids from the page
    #season_teams=season_table.find('a',attrs={'class':'team-link'})
    team_tags=[]         
    for team in season_table:
        #print(team['data-team-id'])
        team_namer=team.find('a',attrs={'class':'team-link'})
        team_name=(team_namer.getText())
        team_tags.append([team['data-team-id'],team_name])      #gets and saves the team name and id and saves it
    driver.delete_all_cookies()
    
    return team_tags
    

In [5]:
#to get all the team players and attributes for a season in the PL
def get_team_roster(teamid,team_name,season,driver):
    base_url='https://www.whoscored.com/Teams/'+str(teamid)+'/Archive/England-' 
    adder=team_name.replace(' ','-')
   
    team_url= base_url+adder #gets to the team homepage
    tag=get_archived_value(team_url,season,driver) #gets the tag address for the season we are looking for
    season_url=team_url+'?stageId='+str(tag)  #link to the particular season
    historical_text=driver.get(season_url)
    xpath1='//*[@id="qcCmpButtons"]/button[2]'   #to get the location of the accept button
     #For maximizing window
    #gives an implicit wait for 5 seconds
    driver.implicitly_wait(5)
    time.sleep(10)
    destination_page_link = driver.find_element_by_xpath(xpath1)
    
    time.sleep(10)
    
    team_id=teamid
    team=team_name
    destination_page_link.click()
    driver.maximize_window()
    data=[]
    for j in range(1,5):
        print(j)
        #if counter==0
        xpath2='//*[@id="team-squad-archive-stats-options"]/li['+str(j)+']/a'  #gets the location of the stats box
        destination_page_link1=driver.find_element_by_xpath(xpath2)
        destination_page_link1.location_once_scrolled_into_view  #scrolls to get the box into view to make it clickable
        driver.delete_all_cookies()
        driver.implicitly_wait(5)
        time.sleep(15)
        destination_page_link1.click()
        time.sleep(15)
        
        
        
        
        soup_historical = BeautifulSoup(driver.page_source, 'lxml')
        historical_table_summary=soup_historical.find_all('tr', attrs={'class':True})
        starter=int(int(len(historical_table_summary))*(j-1)/j)
        print((starter))
        '''This stucture below gets all the stats info for players '''    
        for player in historical_table_summary[starter:]:
            id_finder=player.find('a',href=True)

            if j==1:
                player_atts=[team_id,team]
                player_id=id_finder['href'].split('/')[-3]
                player_name=id_finder['href'].split('/')[-1]
                player_atts.append(player_id)
                player_atts.append(player_name)
                summary_stats=player.getText().replace('-','0').split()[-9:]
                player_atts.extend(summary_stats)
                data.append(player_atts)

            elif j==2:
                player_id=id_finder['href'].split('/')[-3]
                summary_stats=player.getText().replace('-','0').split()[-9:-1]
                for players in data:
                    if player_id==players[2]:
                        players.extend(summary_stats)
                        break
            elif j==3:
                player_id=id_finder['href'].split('/')[-3]
                summary_stats=player.getText().replace('-','0').split()[-7:-1]
                for players in data:
                    if player_id==players[2]:
                        players.extend(summary_stats)
                        break 
            elif j==4:
                player_id=id_finder['href'].split('/')[-3]
                summary_stats=player.getText().replace('-','0').split()[-6:-1]
                for players in data:
                    if player_id==players[2]:
                        players.extend(summary_stats)
                        break
                                
            '''
            link=player['href']
            if link.startswith('#team'):
                link_stats=season_url+link
                driver.get(link_stats)
                soup_stats=BeautifulSoup(driver.page_source, 'lxml')
                stats_players=soup_stats.find_all('tr', class_=True)
                for playerstat in stats_players:
                #print(playerstat.getText())
            
            #print(link_stats)
            '''
    driver.delete_all_cookies()
    
    driver.minimize_window()
    
    return data

In [6]:
def main():
    
    #driver = webdriver.Chrome(r"C:\Users\varun\OneDrive\Documents\classes\CS229\Final_Project\chromedriver.exe")
    seasons=['2009/2010','2010/2011','2011/2012','2012/2013','2013/2014','2014/2015','2015/2016','2016/2017','2017/2018','2018/2019']
    
    for season in seasons[-1:]:
        driver = webdriver.Chrome(r"C:\Users\varun\OneDrive\Documents\classes\CS229\Final_Project\chromedriver.exe")
        #print(season)
        filename='rosters_with_stats_'+season.replace('/','-')+'.csv'
        
        header=['team-id','team','player-id','Player',\
                'Goals', 'Assists', 'Yel', 'Red', 'SpG', 'PS%', 'AerialsWon','MotM','rating',\
                'Tackles', 'Inter', 'Fouls', 'Offsides','Clear', 'Drb', 'Blocks', 'OwnG',\
                 'KeyP', 'Drb', 'Fouled', 'Off', 'Disp','UnsTch',\
                 'AvgP', 'PS%', 'Crosses', 'LongB', 'ThrB']

        season_players=pd.DataFrame(columns=header)

        teams=get_season_teams(season, driver)
        print(teams)
        
        i=0
        for team in teams:
            print(team[1])
            driver = webdriver.Chrome(r"C:\Users\varun\OneDrive\Documents\classes\CS229\Final_Project\chromedriver.exe")
            team_roster_w_stats=get_team_roster(team[0],team[1],season,driver)
            #print(team_roster_w_stats)
            team_panda=pd.DataFrame(team_roster_w_stats,columns=header)

            season_players=pd.concat([season_players,team_panda], ignore_index=True)
            driver.delete_all_cookies()
            driver.quit()

        print(season_players)
        season_players.to_csv(filename, encoding='utf-8', index=False)
    driver.quit()
    
    
    

In [7]:
if __name__ == '__main__':
    main()
    

/Regions/252/Tournaments/2/Seasons/7361/England-Premier-League
[['167', 'Manchester City'], ['26', 'Liverpool'], ['15', 'Chelsea'], ['30', 'Tottenham'], ['13', 'Arsenal'], ['32', 'Manchester United'], ['161', 'Wolverhampton Wanderers'], ['31', 'Everton'], ['14', 'Leicester'], ['29', 'West Ham'], ['27', 'Watford'], ['162', 'Crystal Palace'], ['23', 'Newcastle United'], ['183', 'Bournemouth'], ['184', 'Burnley'], ['18', 'Southampton'], ['211', 'Brighton'], ['188', 'Cardiff'], ['170', 'Fulham'], ['166', 'Huddersfield']]
Manchester City
2018/2019
<class 'str'>
<option value="16368">Premier League - 2018/2019</option>
1
0
2
21
3
42
4
63
Liverpool
2018/2019
<class 'str'>
<option value="16368">Premier League - 2018/2019</option>
1
0
2
23
3
46
4
69
Chelsea
2018/2019
<class 'str'>
<option value="16368">Premier League - 2018/2019</option>
1
0
2
24
3
48
4
72
Tottenham
2018/2019
<class 'str'>
<option value="16368">Premier League - 2018/2019</option>
1
0
2
28
3
56
4
84
Arsenal
2018/2019
<class 'str

In [77]:
x=[[1,2,3,4,5],[6,7,8,9,10]]
obj2=pd.DataFrame(columns=[1,2,3,4,5])
oj2.append[[x],]
header=['team-id','team','player-id','Player',\
            'Goals', 'Assists', 'Yel', 'Red', 'SpG', 'PS%', 'AerialsWon','MotM','rating',\
            'Tackles', 'Inter', 'Fouls', 'Offsides','Clear', 'Drb', 'Blocks', 'OwnG',\
             'KeyP', 'Drb', 'Fouled', 'Off', 'Disp','UnsTch',\
             'AvgP', 'PS%', 'Crosses', 'LongB', 'ThrB']
obj1=pd.DataFrame(columns=header)
print(obj1)

Empty DataFrame
Columns: [team-id, team, player-id, Player, Goals, Assists, Yel, Red, SpG, PS%, AerialsWon, MotM, rating, Tackles, Inter, Fouls, Offsides, Clear, Drb, Blocks, OwnG, KeyP, Drb, Fouled, Off, Disp, UnsTch, AvgP, PS%, Crosses, LongB, ThrB]
Index: []

[0 rows x 32 columns]
